In [1]:
import os
import sys
import pyts
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch

from tsx.perturbation import TimeSeriesPerturbation
from tsx.xai.lime import LIMETimeSeries, XAIModels
from tsx.xai import evaluation as eva

from data_util import *
from viz import *

import itertools
import pandas as pd
from biokit.viz import corrplot

### Objectives
1. Evaluate between different XAI models but same family (Ridge, for example)
1. Evaluate between different DL model, but same XAI model
1. Evaluate the same on Multivariate Time Series
1. (Todo) -> Evaluate the same on Univariate Time Series

### Metrics to Evaluation
1. coef correlations
1. r2-scores
    the scores when building XAI model in approximation process $f(z) ~ g(z') = w * z'$


## Prepare Data Set

In [3]:
independents = ["dew", "temp", "press", "wind_direction", "wind_speed", "snow", "rain"]
dependent = "pollution"

# Load data
df = load_data_set_bejin()
x_scaler, y_scaler = get_xy_scalers(df, independents, dependent)

# Normalize data 
df[independents] = x_scaler.transform(df[independents].values)
df[dependent] = y_scaler.transform(df[dependent].values.reshape(-1, 1))

# Global param
n_steps = 128
window_size = 8
n_variables = len(independents)
samples_size = 100

In [4]:
indices = [i for i in range(len(df) - 129)]
train, test = np.split(indices, [int(len(indices)*0.8)])

X = []
y = []
for index in train:
    X.append(df.iloc[index:index+n_steps][independents].values)
    y.append(df.iloc[index+n_steps][dependent])
    
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)


X_test = []
y_test = []
for index in test:
    X_test.append(df.iloc[index:index+n_steps][independents].values)
    y_test.append(df.iloc[index+n_steps][dependent])
    
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_test.shape, y_test.shape)

(34936, 128, 7) (34936,)
(8735, 128, 7) (8735,)


In [5]:
# from temporalnn.models.temporal import WaveNet
# wavenet = WaveNet().build_model(input_shape=(128, 7),
#                           x_steps=128,
#                           y_steps=1,
#                          gated_activations=['relu', 'sigmoid'],
#                           n_conv_filters=32)
# early_stopper = EarlyStopping(monitor='loss', min_delta=0.01, patience=3, verbose=1)
# wavenet.compile(optimizer='adam', loss='mse')
# wavenet.fit(X, y, callbacks=[early_stopper], epochs=20, batch_size=64, verbose=1)
# wavenet.save("data/wavenet_mts_128_1.h5")

In [6]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dense
# from keras.callbacks import (EarlyStopping, ReduceLROnPlateau)
# lstm = Sequential()
# lstm.add(LSTM(units=50, activation='relu', input_shape=(128, 7)))
# lstm.add(Dense(1))
# lstm.compile(optimizer='adam', loss='mse')

# early_stopper = EarlyStopping(monitor='loss', min_delta=0.002, patience=3, verbose=1)
# lstm.fit(X, y, callbacks=[early_stopper], epochs=20, batch_size=64, verbose=1)
# lstm.save("data/lstm_mts_128_1.h5")

In [7]:
# Models (re-train if something wrong)
import tensorflow as tf
wavenet = tf.keras.models.load_model("data/wavenet_mts_128_1.h5")
lstm = tf.keras.models.load_model("data/lstm_mts_128_1.h5")

In [8]:
def predict_fn(z, model=lstm):
    z_reshaped = z.T.reshape(1, 128, 7)
    z_hat = model.predict(z_reshaped)
    # to avoid zero coef_ for z_hat in[0, 1]
    z_hat = y_scaler.inverse_transform(z_hat.reshape(-1, 1))  
    z_hat = z_hat.ravel()   # z_hat will arround 50 - 150
    return z_hat[0]

def lstm_fn(z):
    return predict_fn(z, model=lstm)

def wavenet_fn(z):
    return predict_fn(z, model=wavenet)

In [9]:
# Prepare Params for different models
scales = ["async", "sync"]
repl_fn = ["zeros", "local_mean", "global_mean"]
model_fn = ["lstm", "wavenet"]

# params = list(itertools.product(scales, ["zeros"], ["lstm"]))
params = list(itertools.product(scales, repl_fn, model_fn))

params_df = pd.DataFrame([{"scale": s, "method": m, "model":model} for s, m, model in params])
# print(params_df)
params_df.style\
    .apply(lambda s: ['background-color: %s' % ('grey' if v else '') for v in s == "async"]) \
    .apply(lambda s: ['background-color: %s' % ('green' if v else '') for v in s == "sync"]) \
    .applymap(lambda s: 'color: %s' % ('cyan' if s == "lstm_fn" else '' )) \
    .applymap(lambda s: 'color: %s' % ('orange' if s == "wavenet_fn" else '' ))

,scale,method,model
0,async,zeros,lstm
1,async,zeros,wavenet
2,async,local_mean,lstm
3,async,local_mean,wavenet
4,async,global_mean,lstm
5,async,global_mean,wavenet
6,sync,zeros,lstm
7,sync,zeros,wavenet
8,sync,local_mean,lstm
9,sync,local_mean,wavenet


In [10]:
def get_xcoef(sample, scale="async", method="zeros", model="lstm", **kwargs):
    lime_ts = LIMETimeSeries(scale=scale, perturb_method=method, **kwargs)
    lime_ts.explain(sample, predict_fn=eval(f"{model}_fn"))
    coef = lime_ts.coef
    x_coef = lime_ts.perturb_obj._x_masked(sample, coef)
    return x_coef

In [11]:
from tqdm.notebook import tqdm
import time
# Generate explanations for each option
# X_test 
# y_test
N = 500
explanations = []
for p in params:   # scale, method, model
    print("Generating explanations for: ", p)

    time.sleep(1)
    # sample_size = 200 if p[0] == 'sync' else 500 
    sample_size = 100   
    exp = [get_xcoef(x.T, *p, samples_size=sample_size, window_size=4) for x in tqdm(X_test[:N])]
    exp = np.array(exp, copy=True)
    d = {"scale": p[0], "method": p[1], "model": p[2], "explanations":exp}
    explanations.append(d)

    time.sleep(1)
    print("Done")

Generating explanations for:  ('async', 'zeros', 'lstm')


  0%|          | 0/500 [00:00<?, ?it/s]

Done
Generating explanations for:  ('async', 'zeros', 'wavenet')


  0%|          | 0/500 [00:00<?, ?it/s]

Done
Generating explanations for:  ('async', 'local_mean', 'lstm')


...

Done
Generating explanations for:  ('sync', 'global_mean', 'lstm')


  0%|          | 0/500 [00:00<?, ?it/s]

Done
Generating explanations for:  ('sync', 'global_mean', 'wavenet')


  0%|          | 0/500 [00:00<?, ?it/s]

Done


In [12]:
# Skip if explanations already generated:
# explanations = generate_explanations()
t = np.array((X_test[:N], y_test[:N], explanations))
np.save('data/explanations_and_test_set_w4.npy',t)

In [13]:
# set 1 -> windows size = 8, sample_size = 200
# set 2 -> windows size = 8, sample size = 100  (not much different in the result)
# set 3 -> windows size = 1, sample size = 100
# set 4 -> windows size = 4, sample size = 100